# Deep learning

## Identifying food type by tensorflow deep learning model : Keras

In [1]:
import os
from os import listdir
from os.path import isfile, join


hot_dog_image_dir = 'input/hot-dog-not-hot-dog/seefood/train/hot_dog'

hot_dog_paths = [join(hot_dog_image_dir, f) for f in listdir(hot_dog_image_dir) if isfile(join(hot_dog_image_dir, f))]
if hot_dog_image_dir + '/.DS_Store' in hot_dog_paths:
    hot_dog_paths.remove(hot_dog_image_dir + '/.DS_Store')
not_hot_dog_image_dir = 'input/hot-dog-not-hot-dog/seefood/train/not_hot_dog'
not_hot_dog_paths = [join(not_hot_dog_image_dir, f) for f in listdir(not_hot_dog_image_dir) if isfile(join(not_hot_dog_image_dir, f))]
if not_hot_dog_image_dir + '/.DS_Store' in not_hot_dog_paths:
    not_hot_dog_paths.remove(not_hot_dog_image_dir + '/.DS_Store')
img_paths = hot_dog_paths + not_hot_dog_paths


### loading data and a pre-trained model, and making predictions unsing ResNet50

In [2]:
from IPython.display import Image, display
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg19 import decode_predictions

Using TensorFlow backend.


In [3]:
image_size = 224

def read_and_prep_images(img_paths, img_height=image_size, img_width=image_size):
    imgs = [load_img(img_path, target_size=(img_height, img_width)) for img_path in img_paths]
    img_array = np.array([img_to_array(img) for img in imgs])
    output = preprocess_input(img_array)
    return(output)

my_model = ResNet50(weights='input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels.h5')
test_data = read_and_prep_images(img_paths)
preds = my_model.predict(test_data)
most_likely_labels = decode_predictions(preds, top=3)

In [4]:
def is_hot_dog(preds):
    is_hot_dog_list=[]
    most_likely_label = decode_predictions(preds, top=1)
    for items in most_likely_label:
        if items[0][1]=='hotdog':
            is_hot_dog_list.append(True)
        else:
            is_hot_dog_list.append(False)
    return(is_hot_dog_list)

### Calculating the accuracy of prediction

In [5]:
def calc_accuracy(model, paths_to_hotdog_images, paths_to_other_images):
    num_hot_dog_images = len(paths_to_hotdog_images)
    num_other_images = len(paths_to_other_images)
    test_data_hotdog = read_and_prep_images(paths_to_hotdog_images)
    preds_hotdog = model.predict(test_data_hotdog)
    labels_y = sum(is_hot_dog(preds_hotdog))
    accuracy = 0
    test_data_not = read_and_prep_images(paths_to_other_images)
    preds_not = model.predict(test_data_not)
    labels_n = num_other_images - sum(is_hot_dog(preds_not))
    total_correct = labels_y + labels_n
    total = num_hot_dog_images + num_other_images
    return(total_correct/total)

# we only apply the model on 50 of the images to speed up the process

my_model_accuracy = calc_accuracy(my_model, hot_dog_paths[:50], not_hot_dog_paths[:50])
print("Fraction correct in the data: {}".format(my_model_accuracy))

Fraction correct in the data: 0.88


### making predictions unsing VGG16 model


In [6]:
from tensorflow.keras.applications import VGG16


vgg16_model = VGG16(weights='input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels.h5')

# we only apply the model on 50 of the images to speed up the process

vgg16_accuracy = calc_accuracy(vgg16_model, hot_dog_paths[:50], not_hot_dog_paths[:50])

print("Fraction correct in the new model: {}".format(vgg16_accuracy))

Fraction correct in the new model: 0.84


## Transfer Learning to apply an existing pretrained model to a new set of images:

### Distinguishing between the sideways and upright images

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 2
resnet_weights_path = 'input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()

# we do not include the prediction layer with setting include_top to False

my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(num_classes, activation='softmax'))

# Indicate not to train the first layer which is already pretrained

my_new_model.layers[0].trainable = False

In [8]:
my_new_model.compile(optimizer='sgd', 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])

In [9]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.logging.set_verbosity(tf.logging.ERROR)

image_size = 224

data_generator = ImageDataGenerator(preprocess_input)


train_generator = data_generator.flow_from_directory(
                                        directory='input/dogs-gone-sideways/images/train',
                                        target_size=(image_size, image_size),
                                        batch_size=10,
                                        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
                                        directory='input/dogs-gone-sideways/images/val',
                                        target_size=(image_size, image_size),
                                        class_mode='categorical')

fit_stats = my_new_model.fit_generator(train_generator,
                                       steps_per_epoch=22,
                                       validation_data=validation_generator,
                                       validation_steps=1)

Found 220 images belonging to 2 classes.
Found 217 images belonging to 2 classes.
Epoch 1/1
22/22 [==============================] - 84s 4s/step - loss: 0.5312 - acc: 0.7136 - val_loss: 0.4227 - val_acc: 0.8125


### Data Augmentation in the imagedatagenerator: fliping horizontally and shifting along width and height of the image

In [11]:
image_size = 224
data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                              horizontal_flip = True,
                                              width_shift_range = 0.1,
                                              height_shift_range = 0.1)
            
data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)

In [12]:

train_generator = data_generator_with_aug.flow_from_directory(
        directory = 'input/dogs-gone-sideways/images/train',
        target_size=(image_size, image_size),
        batch_size=12,
        class_mode='categorical')


validation_generator = data_generator_no_aug.flow_from_directory(
        directory = 'input/dogs-gone-sideways/images/val',
        target_size=(image_size, image_size),
        class_mode='categorical')

my_new_model.fit_generator(
        train_generator,
        epochs = 3,
        steps_per_epoch=19,
        validation_data=validation_generator)

Found 220 images belonging to 2 classes.
Found 217 images belonging to 2 classes.
Epoch 1/3
19/19 [==============================] - 142s 7s/step - loss: 0.5100 - acc: 0.7548 - val_loss: 0.2500 - val_acc: 0.9124
Epoch 2/3
19/19 [==============================] - 125s 7s/step - loss: 0.2073 - acc: 0.9517 - val_loss: 0.1615 - val_acc: 0.9355
Epoch 3/3
19/19 [==============================] - 140s 7s/step - loss: 0.1967 - acc: 0.9341 - val_loss: 0.3051 - val_acc: 0.8618


### Apply deep learning on a set of fashion images from scratch

In [24]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras

img_rows, img_cols = 28, 28
num_classes = 10

# Extract labels and reshape pixel intensity data to 28*28 

def prep_data(raw):
    y = raw[:, 0]
    out_y = keras.utils.to_categorical(y, num_classes)
    
    x = raw[:,1:]
    num_images = raw.shape[0]
    out_x = x.reshape(num_images, img_rows, img_cols, 1)
    out_x = out_x / 255
    return out_x, out_y

fashion_file = "input/fashionmnist/fashion-mnist_train.csv"
fashion_data = np.loadtxt(fashion_file, skiprows=1, delimiter=',')
x, y = prep_data(fashion_data)

In [25]:
fashion_model = Sequential()

### Adding multiple convolutional layers 

In [26]:
# only the first convolutional layer needs to get the input shape

fashion_model.add(Conv2D(12, kernel_size=(3, 3), activation='relu', input_shape=(img_rows, img_cols, 1)))

In [16]:
fashion_model.add(Conv2D(20, kernel_size=(3, 3), activation='relu'))
fashion_model.add(Conv2D(20, kernel_size=(3, 3), activation='relu'))

# convert output of the previous layers to a 1D array

fashion_model.add(Flatten())
fashion_model.add(Dense(100, activation='relu'))
fashion_model.add(Dense(num_classes, activation='softmax'))

In [17]:
fashion_model.compile(optimizer='adam', 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])

In [18]:
fashion_model.fit(x, y,
          batch_size=100,
          epochs=4,
          validation_split = 0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/4
48000/48000 [==============================] - 91s 2ms/step - loss: 0.4731 - acc: 0.8288 - val_loss: 0.3632 - val_acc: 0.8688
Epoch 2/4
48000/48000 [==============================] - 78s 2ms/step - loss: 0.2991 - acc: 0.8909 - val_loss: 0.2923 - val_acc: 0.8927
Epoch 3/4
48000/48000 [==============================] - 77s 2ms/step - loss: 0.2448 - acc: 0.9113 - val_loss: 0.2618 - val_acc: 0.9088
Epoch 4/4
48000/48000 [==============================] - 86s 2ms/step - loss: 0.2021 - acc: 0.9256 - val_loss: 0.2596 - val_acc: 0.9109


### Playing with the setups of the convolution layers and checking the outcome

In [19]:
second_fashion_model = Sequential()
second_fashion_model.add(Conv2D(10, kernel_size=(2, 2),
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))

second_fashion_model.add(Conv2D(20, kernel_size=(2, 2), activation='relu'))
second_fashion_model.add(Flatten())
second_fashion_model.add(Dense(100, activation='relu'))
second_fashion_model.add(Dense(num_classes, activation='softmax'))

second_fashion_model.compile(optimizer='adam', 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])

second_fashion_model.fit(x, y,
          batch_size=100,
          epochs=4,
          validation_split = 0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/4
48000/48000 [==============================] - 37s 766us/step - loss: 0.4756 - acc: 0.8316 - val_loss: 0.3376 - val_acc: 0.8783
Epoch 2/4
48000/48000 [==============================] - 33s 681us/step - loss: 0.3015 - acc: 0.8921 - val_loss: 0.3113 - val_acc: 0.8914
Epoch 3/4
48000/48000 [==============================] - 34s 715us/step - loss: 0.2437 - acc: 0.9126 - val_loss: 0.2811 - val_acc: 0.8982
Epoch 4/4
48000/48000 [==============================] - 38s 792us/step - loss: 0.2011 - acc: 0.9279 - val_loss: 0.2854 - val_acc: 0.9008


### Increasing Stride size  and applying 50% Dropouts to the convolutional layers  to avoid overfittling and increase speed

In [21]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout

batch_size = 16

fashion_model = Sequential()
fashion_model.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))
fashion_model.add(Conv2D(16, (3, 3), activation='relu'))
fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation='relu'))
fashion_model.add(Dense(num_classes, activation='softmax'))

fashion_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

fashion_model.fit(x, y,
          batch_size=batch_size,
          epochs=3,
          validation_split = 0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/3
48000/48000 [==============================] - 88s 2ms/step - loss: 0.4037 - acc: 0.8538 - val_loss: 0.3159 - val_acc: 0.8857
Epoch 2/3
48000/48000 [==============================] - 82s 2ms/step - loss: 0.2578 - acc: 0.9046 - val_loss: 0.2710 - val_acc: 0.8984
Epoch 3/3
48000/48000 [==============================] - 81s 2ms/step - loss: 0.1938 - acc: 0.9284 - val_loss: 0.2686 - val_acc: 0.9041


In [27]:
fashion_model_1 = Sequential()
fashion_model_1.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))
fashion_model_1.add(Dropout(0.5))
fashion_model_1.add(Conv2D(16, (3, 3), activation='relu', strides=2))
fashion_model_1.add(Dropout(0.5))
fashion_model_1.add(Flatten())
fashion_model_1.add(Dense(128, activation='relu'))
fashion_model_1.add(Dense(num_classes, activation='softmax'))

fashion_model_1.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

fashion_model_1.fit(x, y,
          batch_size=batch_size,
          epochs=3,
          validation_split = 0.2) 

Train on 48000 samples, validate on 12000 samples
Epoch 1/3
48000/48000 [==============================] - 47s 978us/step - loss: 0.5476 - acc: 0.7984 - val_loss: 0.3806 - val_acc: 0.8648
Epoch 2/3
48000/48000 [==============================] - 44s 919us/step - loss: 0.3995 - acc: 0.8518 - val_loss: 0.3530 - val_acc: 0.8701
Epoch 3/3
48000/48000 [==============================] - 42s 867us/step - loss: 0.3510 - acc: 0.8681 - val_loss: 0.3020 - val_acc: 0.8891
